# Similarity Ensemble

In [ ]:
%load_ext autoreload

from os import chdir
chdir('../')

In [1]:
from src.utils.loader import *
from scipy.sparse import *
import numpy as np
from src.utils.feature_weighting import *
from src.utils.matrix_utils import compute_cosine, top_k_filtering, yadistance
from src.utils.BaseRecommender import BaseRecommender


class ContentBasedFiltering(BaseRecommender):

    """
    Good conf: tag aggr 3,10; tfidf l1 norm over all matrix
    MAP@5  0.11772497678137457 with 10 shrinkage,
                                    100 k_filtering and other as before
    MAP@5  0.12039006297936491 urm weight 0.7
    MAP@5  0.12109109578826009 without playcount and duration

    Current best:
    CBF (album 1.0, artists 1.0, no duration/playcount)
        + URM 0.8
        + TOP-55 (TFIDF (tags 1.0))
        MAP@5 0.11897304011860126
        Public leaderboard: 0.09616


    """

    def __init__(self, shrinkage=10, k_filtering=100):
        # final matrix of predictions
        self.R_hat = None

        # for keeping reference between playlist and row index
        self.pl_id_list = []
        # for keeping reference between tracks and column index
        self.tr_id_list = []

        self.shrinkage = shrinkage
        self.k_filtering = k_filtering

    def fit(self, urm, target_playlist, target_tracks, dataset, topK_tag=55):
        """
        urm: user rating matrix
        target playlist is a list of playlist id
        target_tracks is a list of track id
        shrinkage: shrinkage factor for significance weighting
        S = ICM' ICM
        R = URM S
        In between eliminate useless row of URM and useless cols of S
        """
        # initialization

        self.pl_id_list = list(target_playlist)
        self.tr_id_list = list(target_tracks)
        self.dataset = dataset
        S = None
        urm = urm.tocsr()
        print("CBF started")
        # get ICM from dataset, assume it already cleaned
        icm = dataset.build_icm()

        # Build the tag matrix, apply TFIDF
        print("Build tags matrix and apply TFIDF...")
        icm_tag = dataset.build_tags_matrix()
        #tag_weight = np.array(urm.dot(icm_tag.transpose()).sum(axis=0)).squeeze()
        #print(tag_weight[tag_weight>5].shape)
        #icm_tag = icm_tag[tag_weight>5].multiply(tag_weight[tag_weight>5])
        tags = applyTFIDF(icm_tag, norm='l1')
        #tags = icm_tag[tag_weight>5]
        # Before stacking tags with the rest of the ICM, we keep only
        # the top K tags for each item. This way we try to reduce the
        # natural noise added by such sparse features.
        tags = top_k_filtering(tags.transpose(), topK=topK_tag).transpose()
        #tags.data = tags.data / np.max(tags.data)
        #newvalue= (max'-min')/(max-min)*(value-max)+max'

        # User augmented UCM
        # print("Building User augmented ICM")
        # ucm = dataset.build_ucm()
        # ua_icm = user_augmented_icm(urm, ucm)
        # ua_icm = top_k_filtering(ua_icm.transpose(), topK=55).transpose()

        # stack all
        icm = vstack([icm, tags, urm * 1], format='csr')
        # icm = vstack([icm, tags, applyTFIDF(urm)], format='csr')

        S = compute_cosine(icm.transpose()[[dataset.get_track_index_from_id(x)
                                            for x in self.tr_id_list]],
                       icm,
                       k_filtering=self.k_filtering,
                       shrinkage=self.shrinkage,
                       n_threads=4,
                       chunksize=1000)
        s_norm = S.sum(axis=1)

        # Normalize S
        S = S.multiply(csr_matrix(np.reciprocal(s_norm)))
        print("Similarity matrix ready!")

        # Keep only the target playlists in the URM
        urm_cleaned = urm[[dataset.get_playlist_index_from_id(x)
                           for x in self.pl_id_list]]
        self.S = S.transpose()

        # Compute ratings
        R_hat = urm_cleaned.dot(S.transpose().tocsc()).tocsr()
        print("R_hat done")

        # Remove the entries in R_hat that are already present in the URM
        urm_cleaned = urm_cleaned[:, [dataset.get_track_index_from_id(x)
                                      for x in self.tr_id_list]]
        R_hat[urm_cleaned.nonzero()] = 0
        R_hat.eliminate_zeros()

        print("Shape of final matrix: ", R_hat.shape)
        self.R_hat = R_hat

    def getW(self):
        """
        Returns the similary matrix with dimensions I x I
        S is IxT
        """
        return self.S.tocsr()

    def predict(self, at=5):
        """
        returns a dictionary of
        'pl_id': ['tr_1', 'tr_at'] for each playlist in target playlist
        """
        recs = {}
        for i in range(0, self.R_hat.shape[0]):
            pl_id = self.pl_id_list[i]
            pl_row = self.R_hat.data[self.R_hat.indptr[i]:
                                     self.R_hat.indptr[i + 1]]
            # get top 5 indeces. argsort, flip and get first at-1 items
            sorted_row_idx = np.flip(pl_row.argsort(), axis=0)[0:at]
            track_cols = [self.R_hat.indices[self.R_hat.indptr[i] + x]
                          for x in sorted_row_idx]
            tracks_ids = [self.tr_id_list[x] for x in track_cols]
            recs[pl_id] = tracks_ids
        return recs

    def getR_hat(self):
        return self.R_hat

    def get_model(self):
        """
        Returns the complete R_hat
        """
        return self.R_hat.copy()


def applyTFIDF(matrix, norm='l1'):
    from sklearn.feature_extraction.text import TfidfTransformer
    transformer = TfidfTransformer(norm=norm, use_idf=True,
                                   smooth_idf=True, sublinear_tf=False)
    tfidf = transformer.fit_transform(matrix.transpose())
    return tfidf.transpose()


def produceCsv():
    # export csv
    dataset = Dataset(load_tags=True,
                      filter_tag=False,
                      weight_tag=False)
    dataset.set_track_attr_weights_2(1.0, 1.0, 0.0, 0.0, 0.0,
                                     1.0, 1.0, 0.0, 0.0)
    cbf_exporter = ContentBasedFiltering()
    urm = dataset.build_train_matrix()
    tg_playlist = list(dataset.target_playlists.keys())
    tg_tracks = list(dataset.target_tracks.keys())
    # Train the model with the best shrinkage found in cross-validation
    cbf_exporter.fit(urm,
                     tg_playlist,
                     tg_tracks,
                     dataset)
    recs = cbf_exporter.predict()
    with open('submission_cbf.csv', mode='w', newline='') as out:
        fieldnames = ['playlist_id', 'track_ids']
        writer = csv.DictWriter(out, fieldnames=fieldnames, delimiter=',')
        writer.writeheader()
        for k in tg_playlist:
            track_ids = ''
            for r in recs[k]:
                track_ids = track_ids + r + ' '
            writer.writerow({'playlist_id': k,
                             'track_ids': track_ids[:-1]})


def evaluateMap():
    from src.utils.evaluator import Evaluator
    dataset = Dataset(load_tags=True,
                      filter_tag=False,
                      weight_tag=False)
    dataset.set_track_attr_weights_2(1.0, 1.0, 0.0, 0.0, 0.0,
                                     1.0, 1.0, 0.0, 0.0)
    # seed = 0xcafebabe
    # print("Evaluating with initial seed: {}".format(seed))
    ev = Evaluator(seed=False)
    ev.cross_validation(5, dataset.train_final.copy())
    cbf = ContentBasedFiltering()
    for i in range(0, 5):
        urm, tg_tracks, tg_playlist = ev.get_fold(dataset)
        cbf.fit(urm,
                list(tg_playlist),
                list(tg_tracks),
                dataset)
        recs = cbf.predict()
        ev.evaluate_fold(recs)

    map_at_five = ev.get_mean_map()
    print("MAP@5 ", map_at_five)


def crossValidation():
    from src.utils.evaluator import Evaluator
    pass

# Item Based Similarity

In [2]:
def ibs(urm, dataset):
    urm_sim = applyTFIDF(urm)
    #urm_sim = top_k_filtering(urm_sim.transpose()).transpose()
    S = compute_cosine(urm_sim.transpose()[[dataset.get_track_index_from_id(x)
                                    for x in tg_tracks]],
                       urm_sim,
                       k_filtering=100,
                       shrinkage=10,
                       n_threads=4,
                       chunksize=1000)
    s_norm = S.sum(axis=1)
    s_norm[s_norm==0] = 1
    # Normalize S
    S = S.multiply(csr_matrix(np.reciprocal(s_norm)))
    print("Similarity matrix ready!")
    return S.transpose()

# User Based Similarity

In [3]:
def ubs(urm, ucm, dataset):
    ucm = dataset.build_ucm()
    iucm = ucm.dot(urm)
    iucm = applyTFIDF(iucm)
    iucm = top_k_filtering(iucm.transpose(), 100).transpose()
    S = compute_cosine(iucm.transpose()[[dataset.get_track_index_from_id(x)
                                                for x in tg_tracks]],
                           iucm,
                           k_filtering=100,
                           shrinkage=10,
                           n_threads=4,
                           chunksize=250)
    s_norm = S.sum(axis=1)
    s_norm[s_norm==0] = 1
    print("S done")
    # Normalize S
    S = S.multiply(csr_matrix(np.reciprocal(s_norm)))
    print("Similarity matrix ready!")
    return S.transpose()

# Only Content Based

In [3]:
def icm_sim(icm, dataset):
    icm_tag = dataset.build_tags_matrix()
    #tag_weight = np.array(urm.dot(icm_tag.transpose()).sum(axis=0)).squeeze()
    #print(tag_weight[tag_weight>5].shape)
    #icm_tag = icm_tag[tag_weight>5].multiply(tag_weight[tag_weight>5])
    tags = applyTFIDF(icm_tag, norm='l1')
    #tags = icm_tag[tag_weight>5]
    # Before stacking tags with the rest of the ICM, we keep only
    # the top K tags for each item. This way we try to reduce the
    # natural noise added by such sparse features.
    tags = top_k_filtering(tags.transpose(), topK=55).transpose()
    #tags.data = tags.data / np.max(tags.data)
    #newvalue= (max'-min')/(max-min)*(value-max)+max'

    # User augmented UCM
    # print("Building User augmented ICM")
    # ucm = dataset.build_ucm()
    # ua_icm = user_augmented_icm(urm, ucm)
    # ua_icm = top_k_filtering(ua_icm.transpose(), topK=55).transpose()

    # stack all
    icm = vstack([icm, tags*0.4], format='csr')
    #icm = applyTFIDF(icm)
    #icm = top_k_filtering(icm.transpose(), topK=200).transpose()
    # icm = vstack([icm, tags, applyTFIDF(urm)], format='csr')

    S = compute_cosine(icm.transpose()[[dataset.get_track_index_from_id(x)
                                        for x in tg_tracks]],
                   icm,
                   k_filtering=100,
                   shrinkage=10,
                   n_threads=4,
                   chunksize=1000)
    s_norm = S.sum(axis=1)
    s_norm[s_norm==0] = 1
    print("S done")
    # Normalize S
    S = S.multiply(csr_matrix(np.reciprocal(s_norm)))
    print("Similarity matrix ready!")
    return S.transpose()

# Initialization

In [22]:
from src.utils.evaluator import Evaluator
from src.utils.loader import *
print("started")
print("hey")
dataset = Dataset(load_tags=True,
                  filter_tag=False,
                  weight_tag=False)
dataset.set_track_attr_weights_2(1.5, 1.6, 0, 0, 0.0,
                                 1.0, 0, 0.0, 0.0)
# seed = 0xcafebabe
# print("Evaluating with initial seed: {}".format(seed))
ev = Evaluator(seed=False)
ev.cross_validation(5, dataset.train_final.copy())
urm, tg_tracks, tg_playlist = ev.get_fold(dataset)
print("Done")

started
hey
File found, retrieving urm from it.
Load from file takes 2.57 seconds
Done


# Ensemble

### SLIM BPR

In [23]:
%autoreload 2

from src.ML.BPRSLIM import BPRSLIM
from src.utils.matrix_utils import applyTfIdf, top_k_filtering

import os
dataset.set_track_attr_weights_2(1, 1, 0, 0, 0, 0, 0, 0, 0)

print('Building the ICM for SLIMBPR...')
icm = dataset.build_icm()

icm_tag = dataset.build_tags_matrix()

tags = applyTfIdf(icm_tag)

# Before stacking tags with the rest of the ICM, we keep only
# the top K tags for each item. This way we try to reduce the
# natural noise added by such sparse features.
tags = top_k_filtering(tags.transpose(), topK=55).transpose()
tags.data = np.ones_like(tags.data)

# stack all
icm = vstack([icm, tags], format='csr')

Building the ICM for SLIMBPR...


In [24]:
slim = BPRSLIM(epochs=30,
               epochMultiplier=1.0,
               sgd_mode='rmsprop',
               learning_rate=5e-05,
               topK=300,
               urmSamplingChances=1 / 5,
               icmSamplingChances=4 / 5)

In [25]:
slim.set_evaluation_every(5, ev)
slim.fit(urm.tocsr(),
         icm.tocsr(),
         tg_playlist,
         tg_tracks,
         dataset)

S_bprslim = slim.getParameters()
S_bprslim = S_bprslim[:,[dataset.get_track_index_from_id(x) for x in tg_tracks]]

# keep only 100 elements
S_bprslim = top_k_filtering(S_bprslim.transpose(), 100).transpose()

# normalize
s_norm = S_bprslim.sum(axis=0)
s_norm[s_norm==0] = 1
print("S done")

S_bprslim = S_bprslim.multiply(csr_matrix(np.reciprocal(s_norm)))

Running fit process.
Processed 500000 ( 29.85% ) in 276.14 seconds. Sample per second: 1811
Processed 1000000 ( 59.70% ) in 39.12 seconds. Sample per second: 3173
Processed 1500000 ( 89.54% ) in 23.59 seconds. Sample per second: 4430
Processed 1675142 ( 100.00% ) in 9.75 seconds. Sample per second: 4817
Return S matrix to python caller...
Epoch 0 of 30 complete in 6.20 minutes
Processed 500000 ( 29.85% ) in 14.14 seconds. Sample per second: 35367
Processed 1000000 ( 59.70% ) in 11.08 seconds. Sample per second: 39875
Processed 1500000 ( 89.54% ) in 10.86 seconds. Sample per second: 41832
Processed 1675142 ( 100.00% ) in 4.65 seconds. Sample per second: 42245
Return S matrix to python caller...
Epoch 1 of 30 complete in 1.31 minutes
Processed 500000 ( 29.85% ) in 22.95 seconds. Sample per second: 21789
Processed 1000000 ( 59.70% ) in 12.39 seconds. Sample per second: 29080
Processed 1500000 ( 89.54% ) in 10.87 seconds. Sample per second: 33433
Processed 1675142 ( 100.00% ) in 4.80 secon

/media/leonardoarcari/Data/Home/Development/python_projects/recsys_challenge/lib/python3.6/site-packages/scipy/sparse/compressed.py:742: SparseEfficiencyWarning: Changing the sparsity structure of a csr_matrix is expensive. lil_matrix is more efficient.
  SparseEfficiencyWarning)


MAP@5: 0.09663650757747115
Epoch 5 of 30 complete in 3.36 minutes
Processed 500000 ( 29.85% ) in 19.78 seconds. Sample per second: 25282
Processed 1000000 ( 59.70% ) in 13.31 seconds. Sample per second: 30954
Processed 1500000 ( 89.54% ) in 11.09 seconds. Sample per second: 34812
Processed 1675142 ( 100.00% ) in 4.24 seconds. Sample per second: 35461
Return S matrix to python caller...
Epoch 6 of 30 complete in 1.98 minutes
Processed 500000 ( 29.85% ) in 36.57 seconds. Sample per second: 13674
Processed 1000000 ( 59.70% ) in 13.04 seconds. Sample per second: 20393
Processed 1500000 ( 89.54% ) in 11.97 seconds. Sample per second: 24602
Processed 1675142 ( 100.00% ) in 5.08 seconds. Sample per second: 25740
Return S matrix to python caller...
Epoch 7 of 30 complete in 1.79 minutes
Processed 500000 ( 29.85% ) in 27.27 seconds. Sample per second: 18334
Processed 1000000 ( 59.70% ) in 13.62 seconds. Sample per second: 24620
Processed 1500000 ( 89.54% ) in 12.11 seconds. Sample per second: 2

Return S matrix to python caller...
Epoch 27 of 30 complete in 1.84 minutes
Processed 500000 ( 29.85% ) in 24.94 seconds. Sample per second: 20046
Processed 1000000 ( 59.70% ) in 12.30 seconds. Sample per second: 27545
Processed 1500000 ( 89.54% ) in 11.42 seconds. Sample per second: 31634
Processed 1675142 ( 100.00% ) in 4.19 seconds. Sample per second: 32726
Return S matrix to python caller...
Epoch 28 of 30 complete in 1.57 minutes
Processed 500000 ( 29.85% ) in 23.71 seconds. Sample per second: 21084
Processed 1000000 ( 59.70% ) in 12.98 seconds. Sample per second: 27791
Processed 1500000 ( 89.54% ) in 11.81 seconds. Sample per second: 32046
Processed 1675142 ( 100.00% ) in 4.75 seconds. Sample per second: 33010
Return S matrix to python caller...
Epoch 29 of 30 complete in 1.55 minutes
Fit completed in 57.35 minutes
S done


### Matrix Factorization

In [13]:
from src.CBF.CBF_MF import ContentBasedFiltering as CBF
from src.MF.MF_BPR.MF_BPR import MF_BPR

# Init ICM params
dataset.set_track_attr_weights_2(1.5, 1.6, 0, 0, 0.0,
                                 1.0, 0, 0.0, 0.0)
cbf = CBF()
    
cbf.fit(urm, tg_playlist,
        tg_tracks,
        dataset)

# get R_hat
R_hat_aug = cbf.getR_hat()
print(R_hat_aug.nnz)

mf = MF_BPR()

CBF started
Build tags matrix and apply TFIDF...
Running 4 workers...
[ 5418 ] Building cosine similarity matrix for [0, 1000)...
[ 5419 ] Building cosine similarity matrix for [25000, 26000)...
[ 5420 ] Building cosine similarity matrix for [50000, 51000)...
[ 5421 ] Building cosine similarity matrix for [75000, 76000)...
[ 5418 ] Building cosine similarity matrix for [1000, 2000)...
[ 5419 ] Building cosine similarity matrix for [26000, 27000)...
[ 5420 ] Building cosine similarity matrix for [51000, 52000)...
[ 5421 ] Building cosine similarity matrix for [76000, 77000)...
[ 5419 ] Building cosine similarity matrix for [27000, 28000)...
[ 5418 ] Building cosine similarity matrix for [2000, 3000)...
[ 5420 ] Building cosine similarity matrix for [52000, 53000)...
[ 5421 ] Building cosine similarity matrix for [77000, 78000)...
[ 5419 ] Building cosine similarity matrix for [28000, 29000)...
[ 5418 ] Building cosine similarity matrix for [3000, 4000)...
[ 5420 ] Building cosine simila

/media/leonardoarcari/Data/Home/Development/python_projects/recsys_challenge/lib/python3.6/site-packages/scipy/sparse/compressed.py:742: SparseEfficiencyWarning: Changing the sparsity structure of a csr_matrix is expensive. lil_matrix is more efficient.
  SparseEfficiencyWarning)


Shape of final matrix:  (57561, 100000)
1246603


In [14]:
# MAP@5: 0.08256503053607782 with 500 factors after 10 epochs
# MAP@5: 0.08594586443489391 with 500 factors afetr 4 epochs no_components=500, epoch_multiplier=2, l_rate=1e-2
mf.fit(R_hat_aug, dataset, list(tg_playlist), list(tg_tracks), n_epochs=1, no_components=500, epoch_multiplier=6, l_rate=1e-2, use_icm=False)
recs = mf.predict_dot_custom(urm)
ev.evaluate_fold(recs)

# MAP@5: 0.09407901681369218 with neighborhood
# MAP@5: 0.09854105406016736 with neighborhood after 4 epochs
recs = mf.predict_knn_custom(urm)
ev.evaluate_fold(recs)

S_mf = mf.S

Processed 50000 ( 0.67% ) in 8.88 seconds. Sample per second: 5631
Processed 100000 ( 1.34% ) in 14.69 seconds. Sample per second: 4407
Processed 150000 ( 2.01% ) in 18.51 seconds. Sample per second: 3702
Processed 200000 ( 2.67% ) in 20.64 seconds. Sample per second: 3298
Processed 250000 ( 3.34% ) in 21.91 seconds. Sample per second: 3052
Processed 300000 ( 4.01% ) in 22.78 seconds. Sample per second: 2891
Processed 350000 ( 4.68% ) in 22.95 seconds. Sample per second: 2779
Processed 400000 ( 5.35% ) in 23.41 seconds. Sample per second: 2695
Processed 450000 ( 6.02% ) in 23.48 seconds. Sample per second: 2624
Processed 500000 ( 6.68% ) in 22.97 seconds. Sample per second: 2578
Processed 550000 ( 7.35% ) in 23.44 seconds. Sample per second: 2541
Processed 600000 ( 8.02% ) in 22.93 seconds. Sample per second: 2511
Processed 650000 ( 8.69% ) in 23.55 seconds. Sample per second: 2485
Processed 700000 ( 9.36% ) in 23.06 seconds. Sample per second: 2464
Processed 750000 ( 10.03% ) in 22.59

Processed 5850000 ( 78.21% ) in 23.02 seconds. Sample per second: 2241
Processed 5900000 ( 78.88% ) in 22.56 seconds. Sample per second: 2240
Processed 5950000 ( 79.55% ) in 23.12 seconds. Sample per second: 2240
Processed 6000000 ( 80.22% ) in 22.81 seconds. Sample per second: 2240
Processed 6050000 ( 80.89% ) in 23.36 seconds. Sample per second: 2240
Processed 6100000 ( 81.55% ) in 22.90 seconds. Sample per second: 2239
Processed 6150000 ( 82.22% ) in 23.47 seconds. Sample per second: 2239
Processed 6200000 ( 82.89% ) in 23.17 seconds. Sample per second: 2239
Processed 6250000 ( 83.56% ) in 22.84 seconds. Sample per second: 2239
Processed 6300000 ( 84.23% ) in 23.39 seconds. Sample per second: 2238
Processed 6350000 ( 84.90% ) in 22.96 seconds. Sample per second: 2238
Processed 6400000 ( 85.57% ) in 23.53 seconds. Sample per second: 2238
Processed 6450000 ( 86.23% ) in 23.39 seconds. Sample per second: 2238
Processed 6500000 ( 86.90% ) in 23.10 seconds. Sample per second: 2237
Proces

### Full Content Based Filtering

In [36]:
print("ok")
# Initiliaze Models
# dataset.set_track_attr_weights_2(1.5, 1.6, 0.0, 0.0, 0.0,
#                                  1.0, 0.0, 0.0, 0.0)
dataset.set_track_attr_weights_2(1.8272547701242121, 1.9097988118972682, 0.0, 0.0, 0.0,
                                 1.0, 0.0, 0.0, 0.0)
cbf = ContentBasedFiltering()
cbf.fit(urm * 0.7840544888432146,
        list(tg_playlist),
        list(tg_tracks),
        dataset, 
        topK_tag=55)
#recs = cbf.predict()
#ev.evaluate_fold(recs)
S_cbf_full = cbf.S

ok
CBF started
Build tags matrix and apply TFIDF...
Running 4 workers...
[ 6701 ] Building cosine similarity matrix for [4291, 5291)...
[ 6702 ] Building cosine similarity matrix for [8582, 9582)...
[ 6700 ] Building cosine similarity matrix for [0, 1000)...
[ 6703 ] Building cosine similarity matrix for [12873, 13873)...
[ 6702 ] Building cosine similarity matrix for [9582, 10582)...
[ 6700 ] Building cosine similarity matrix for [1000, 2000)...
[ 6703 ] Building cosine similarity matrix for [13873, 14873)...
[ 6701 ] Building cosine similarity matrix for [5291, 6291)...
[ 6703 ] Building cosine similarity matrix for [14873, 15873)...
[ 6702 ] Building cosine similarity matrix for [10582, 11582)...
[ 6700 ] Building cosine similarity matrix for [2000, 3000)...
[ 6701 ] Building cosine similarity matrix for [6291, 7291)...
[ 6703 ] Building cosine similarity matrix for [15873, 16873)...
[ 6700 ] Building cosine similarity matrix for [3000, 4000)...
[ 6701 ] Building cosine similarity m

### URM Item based filtering

In [27]:
S_urm = ibs(urm, dataset)

Running 4 workers...
[ 28695 ] Building cosine similarity matrix for [0, 1000)...
[ 28696 ] Building cosine similarity matrix for [4291, 5291)...
[ 28697 ] Building cosine similarity matrix for [8582, 9582)...
[ 28698 ] Building cosine similarity matrix for [12873, 13873)...
[ 28698 ] Building cosine similarity matrix for [13873, 14873)...
[ 28696 ] Building cosine similarity matrix for [5291, 6291)...
[ 28697 ] Building cosine similarity matrix for [9582, 10582)...
[ 28695 ] Building cosine similarity matrix for [1000, 2000)...
[ 28696 ] Building cosine similarity matrix for [6291, 7291)...
[ 28698 ] Building cosine similarity matrix for [14873, 15873)...
[ 28697 ] Building cosine similarity matrix for [10582, 11582)...
[ 28695 ] Building cosine similarity matrix for [2000, 3000)...
[ 28697 ] Building cosine similarity matrix for [11582, 12582)...
[ 28696 ] Building cosine similarity matrix for [7291, 8291)...
[ 28698 ] Building cosine similarity matrix for [15873, 16873)...
[ 28695 ]

### Only ICM Content based filtering

In [34]:
dataset.set_track_attr_weights_2(1.3777282922511418, 1.3936146985720337, 0.0, 0.0, 0.0,
                                 1.0, 0.0, 0.0, 0.0)
S_icm = icm_sim(dataset.build_icm(), dataset)

Running 4 workers...
[ 594 ] Building cosine similarity matrix for [0, 1000)...
[ 595 ] Building cosine similarity matrix for [4291, 5291)...
[ 596 ] Building cosine similarity matrix for [8582, 9582)...
[ 597 ] Building cosine similarity matrix for [12873, 13873)...
[ 597 ] Building cosine similarity matrix for [13873, 14873)...
[ 596 ] Building cosine similarity matrix for [9582, 10582)...
[ 594 ] Building cosine similarity matrix for [1000, 2000)...
[ 595 ] Building cosine similarity matrix for [5291, 6291)...
[ 595 ] Building cosine similarity matrix for [6291, 7291)...
[ 596 ] Building cosine similarity matrix for [10582, 11582)...
[ 594 ] Building cosine similarity matrix for [2000, 3000)...
[ 597 ] Building cosine similarity matrix for [14873, 15873)...
[ 595 ] Building cosine similarity matrix for [7291, 8291)...
[ 594 ] Building cosine similarity matrix for [3000, 4000)...
[ 596 ] Building cosine similarity matrix for [11582, 12582)...
[ 597 ] Building cosine similarity matrix

### User Content Based Filtering

In [12]:
dataset.set_playlist_attr_weights(1, 1, 1, 0, 0)
ucm = dataset.build_ucm()
S_user = ubs(urm, ucm, dataset)

Running 4 workers...
[ 11167 ] Building cosine similarity matrix for [0, 250)...
[ 11168 ] Building cosine similarity matrix for [4293, 4543)...
[ 11169 ] Building cosine similarity matrix for [8586, 8836)...
[ 11170 ] Building cosine similarity matrix for [12879, 13129)...
[ 11167 ] Building cosine similarity matrix for [250, 500)...
[ 11168 ] Building cosine similarity matrix for [4543, 4793)...
[ 11170 ] Building cosine similarity matrix for [13129, 13379)...
[ 11169 ] Building cosine similarity matrix for [8836, 9086)...
[ 11168 ] Building cosine similarity matrix for [4793, 5043)...
[ 11167 ] Building cosine similarity matrix for [500, 750)...
[ 11170 ] Building cosine similarity matrix for [13379, 13629)...
[ 11169 ] Building cosine similarity matrix for [9086, 9336)...
[ 11168 ] Building cosine similarity matrix for [5043, 5293)...
[ 11167 ] Building cosine similarity matrix for [750, 1000)...
[ 11170 ] Building cosine similarity matrix for [13629, 13879)...
[ 11169 ] Building c

In [29]:
def predict(R_hat, pl_id_list, tr_id_list, at=5):
        """
        returns a dictionary of
        'pl_id': ['tr_1', 'tr_at'] for each playlist in target playlist
        """
        recs = {}
        for i in range(0, R_hat.shape[0]):
            pl_id = pl_id_list[i]
            pl_row = R_hat.data[R_hat.indptr[i]:
                                     R_hat.indptr[i + 1]]
            # get top 5 indeces. argsort, flip and get first at-1 items
            sorted_row_idx = np.flip(pl_row.argsort(), axis=0)[0:at]
            track_cols = [R_hat.indices[R_hat.indptr[i] + x]
                          for x in sorted_row_idx]
            tracks_ids = [tr_id_list[x] for x in track_cols]
            recs[pl_id] = tracks_ids
        return recs

# Bayesian opt

In [6]:
def mixS(params):
    global S_icm, S_user, S_urm, S_cbf_full, S_bprslim, S_mf, urm, tg_playlist, evaluator
    print(params)

    S = S_icm * params[0] + S_urm * params[1] + S_cbf_full * params[2] + S_bprslim * params[3] + S_mf * params[4]
    print("Similarity matrix ready!")
    
    S = top_k_filtering(S.transpose(), topK=300).transpose()

    s_norm = S.sum(axis=0)
    s_norm[s_norm==0] = 1
    # Normalize S
    S= S.multiply(csr_matrix(np.reciprocal(s_norm)))
    
    print("Similarity matrix ready!")

    # Keep only the target playlists in the URM
    urm_cleaned = urm[[dataset.get_playlist_index_from_id(x)
                       for x in tg_playlist]]

    # Compute ratings
    R_hat = urm_cleaned.dot(S.tocsc()).tocsr()
    print("R_hat done")

    # Remove the entries in R_hat that are already present in the URM
    urm_cleaned = urm_cleaned[:, [dataset.get_track_index_from_id(x)
                                  for x in tg_tracks]]
    R_hat[urm_cleaned.nonzero()] = 0
    R_hat.eliminate_zeros()

    print("Shape of final matrix: ", R_hat.shape)
    recs = predict(R_hat, list(tg_playlist), list(tg_tracks))
    map_at_five = ev.evaluate_fold(recs)
    return -map_at_five

In [18]:
# BEST with ICM, URM, CBF_FULL, SLIMBPR
# x = [0.010884732743304261, 0.02719900735954462, 0.048684679013303325, 0.022415631857985566]
from skopt import forest_minimize
from skopt.space import Integer, Real
space = [Real(0.0, 1.0),  # ICM
         Real(0.0, 1.0),  # URM
         Real(0.0, 1.0),  # CBF_FULL
         Real(0.0, 1.0),  # SLIMBPR
         Real(0.0, 1.0)  # MF BPR
        ]
x0 = [1, 0, 0, 0, 0]
x1 = [0, 1, 0, 0, 0]
x2 = [0, 0, 1, 0, 0]
x3 = [0, 0, 0, 1, 0]
x4 = [0, 0, 0, 0, 1]
x5 = [0.88841953253529427, 0.16253614982055045, 0.29397344943800269, 0.047477757893935171, 0.81730840097669222]
x6 = [0.5677248728726197, 0.5518258064369265, 0.9622603220708489, 0.11921505196603724, 0.12842883273924657]
x0s = [x0, x1, x2, x3, x4, x5]
res = forest_minimize(mixS, space, x0=x0s, verbose=True, n_random_starts=50, n_calls=1000, n_jobs=-1)
print('Maximimum p@k found: {:6.5f}'.format(-res.fun))
print('Optimal parameters:')
params = ['ICM', 'URM', 'CBF_FULL', 'SLIMBPR', 'MFBPR']
for (p, x_) in zip(params, res.x):
    print('{}: {}'.format(p, x_))

Iteration No: 1 started. Evaluating function at provided point.
[1, 0, 0, 0, 0]
Similarity matrix ready!
Similarity matrix ready!
R_hat done
Shape of final matrix:  (4421, 17296)
MAP@5: 0.10892332051572054
[0, 1, 0, 0, 0]
Similarity matrix ready!
Similarity matrix ready!
R_hat done
Shape of final matrix:  (4421, 17296)
MAP@5: 0.08958832843248145
[0, 0, 1, 0, 0]
Similarity matrix ready!
Similarity matrix ready!
R_hat done
Shape of final matrix:  (4421, 17296)
MAP@5: 0.11967277388222884
[0, 0, 0, 1, 0]
Similarity matrix ready!
Similarity matrix ready!
R_hat done


/media/leonardoarcari/Data/Home/Development/python_projects/recsys_challenge/lib/python3.6/site-packages/scipy/sparse/compressed.py:742: SparseEfficiencyWarning: Changing the sparsity structure of a csr_matrix is expensive. lil_matrix is more efficient.
  SparseEfficiencyWarning)


Shape of final matrix:  (4421, 17296)
MAP@5: 0.11239689361381285
[0, 0, 0, 0, 1]
Similarity matrix ready!
Similarity matrix ready!
R_hat done
Shape of final matrix:  (4421, 17296)
MAP@5: 0.0927625725703084
[0.8884195325352943, 0.16253614982055045, 0.2939734494380027, 0.04747775789393517, 0.8173084009766922]
Similarity matrix ready!
Similarity matrix ready!
R_hat done
Shape of final matrix:  (4421, 17296)
MAP@5: 0.1153653019678808
Iteration No: 1 ended. Evaluation done at provided point.
Time taken: 144.5887
Function value obtained: -0.1154
Current minimum: -0.1197
Iteration No: 2 started. Evaluating function at provided point.
[0.92930077609532791, 0.70151243435327804, 0.69941451776325325, 0.53742583388359888, 0.44668116046866146]
Similarity matrix ready!
Similarity matrix ready!
R_hat done
Shape of final matrix:  (4421, 17296)
MAP@5: 0.12131870617507377
Iteration No: 2 ended. Evaluation done at provided point.
Time taken: 24.7137
Function value obtained: -0.1213
Current minimum: -0.12

Similarity matrix ready!
Similarity matrix ready!
R_hat done
Shape of final matrix:  (4421, 17296)
MAP@5: 0.12038528236447285
Iteration No: 20 ended. Evaluation done at random point.
Time taken: 24.7235
Function value obtained: -0.1204
Current minimum: -0.1214
Iteration No: 21 started. Evaluating function at random point.
[0.30743463201930871, 0.35639076847131707, 0.87165009166373575, 0.15342155825699214, 0.68862149481011525]
Similarity matrix ready!
Similarity matrix ready!
R_hat done
Shape of final matrix:  (4421, 17296)
MAP@5: 0.11942773128251537
Iteration No: 21 ended. Evaluation done at random point.
Time taken: 25.0694
Function value obtained: -0.1194
Current minimum: -0.1214
Iteration No: 22 started. Evaluating function at random point.
[0.20498165199836388, 0.29822433345967619, 0.083828751861399056, 0.62894486399486849, 0.54396681011975012]
Similarity matrix ready!
Similarity matrix ready!
R_hat done
Shape of final matrix:  (4421, 17296)
MAP@5: 0.11891728869788153
Iteration No:

Similarity matrix ready!
R_hat done
Shape of final matrix:  (4421, 17296)
MAP@5: 0.11958832843248163
Iteration No: 39 ended. Evaluation done at random point.
Time taken: 21.7261
Function value obtained: -0.1196
Current minimum: -0.1221
Iteration No: 40 started. Evaluating function at random point.
[0.034818448845357793, 0.86967421701158409, 0.21462147161317319, 0.36186101840446255, 0.33062867432075421]
Similarity matrix ready!
Similarity matrix ready!
R_hat done
Shape of final matrix:  (4421, 17296)
MAP@5: 0.11689813767624245
Iteration No: 40 ended. Evaluation done at random point.
Time taken: 21.9403
Function value obtained: -0.1169
Current minimum: -0.1221
Iteration No: 41 started. Evaluating function at random point.
[0.90277762490525781, 0.13083185794007915, 0.17753064018678258, 0.97277891764486379, 0.7325977225502226]
Similarity matrix ready!
Similarity matrix ready!
R_hat done
Shape of final matrix:  (4421, 17296)
MAP@5: 0.11737012742215197
Iteration No: 41 ended. Evaluation done

...

Similarity matrix ready!
Similarity matrix ready!
R_hat done
Shape of final matrix:  (4421, 17296)
MAP@5: 0.10417854180803751
Iteration No: 913 ended. Search finished for the next optimal point.
Time taken: 22.9363
Function value obtained: -0.1042
Current minimum: -0.1221
Iteration No: 914 started. Searching for the next optimal point.
[0.00984416304970759, 0.038611070666668364, 0.77109688269564503, 0.0023284654571462675, 0.51850885903122612]
Similarity matrix ready!
Similarity matrix ready!
R_hat done
Shape of final matrix:  (4421, 17296)
MAP@5: 0.11733846037849667
Iteration No: 914 ended. Search finished for the next optimal point.
Time taken: 22.6847
Function value obtained: -0.1173
Current minimum: -0.1221
Iteration No: 915 started. Searching for the next optimal point.
[0.4294515545713099, 0.97803686075069085, 0.40248258017477967, 0.10890689267697498, 0.63530738861585756]
Similarity matrix ready!
Similarity matrix ready!
R_hat done
Shape of final matrix:  (4421, 17296)
MAP@5: 0.11

Similarity matrix ready!
Similarity matrix ready!
R_hat done
Shape of final matrix:  (4421, 17296)
MAP@5: 0.11818291487597112
Iteration No: 932 ended. Search finished for the next optimal point.
Time taken: 22.4851
Function value obtained: -0.1182
Current minimum: -0.1221
Iteration No: 933 started. Searching for the next optimal point.
[0.0004929797395075709, 0.66731252179832901, 0.15619001129924917, 0.11193643717775184, 0.013037381556949782]
Similarity matrix ready!
Similarity matrix ready!
R_hat done
Shape of final matrix:  (4421, 17296)
MAP@5: 0.11068461132473821
Iteration No: 933 ended. Search finished for the next optimal point.
Time taken: 22.6692
Function value obtained: -0.1107
Current minimum: -0.1221
Iteration No: 934 started. Searching for the next optimal point.
[0.33116964114569097, 0.89541791832593665, 0.27858834560977946, 0.0087340940974376408, 0.76345458491321905]
Similarity matrix ready!
Similarity matrix ready!
R_hat done
Shape of final matrix:  (4421, 17296)
MAP@5: 0

Similarity matrix ready!
Similarity matrix ready!
R_hat done
Shape of final matrix:  (4421, 17296)
MAP@5: 0.11340043730679356
Iteration No: 951 ended. Search finished for the next optimal point.
Time taken: 22.7957
Function value obtained: -0.1134
Current minimum: -0.1221
Iteration No: 952 started. Searching for the next optimal point.
[0.0022741819698278713, 0.3134665538497276, 0.24840949719790489, 0.030823822406824693, 0.85398837424911034]
Similarity matrix ready!
Similarity matrix ready!
R_hat done
Shape of final matrix:  (4421, 17296)
MAP@5: 0.11011762044786237
Iteration No: 952 ended. Search finished for the next optimal point.
Time taken: 22.6114
Function value obtained: -0.1101
Current minimum: -0.1221
Iteration No: 953 started. Searching for the next optimal point.
[0.0027762006673779376, 0.3308586548949865, 0.15414329634735516, 0.037997803403477144, 0.56848590041665026]
Similarity matrix ready!
Similarity matrix ready!
R_hat done
Shape of final matrix:  (4421, 17296)
MAP@5: 0.

Similarity matrix ready!
Similarity matrix ready!
R_hat done
Shape of final matrix:  (4421, 17296)
MAP@5: 0.1135512327527709
Iteration No: 970 ended. Search finished for the next optimal point.
Time taken: 22.6865
Function value obtained: -0.1136
Current minimum: -0.1221
Iteration No: 971 started. Searching for the next optimal point.
[0.045402368961913638, 0.46815743444310565, 0.12907533013551045, 0.073726470765324029, 0.036437800413030359]
Similarity matrix ready!
Similarity matrix ready!
R_hat done
Shape of final matrix:  (4421, 17296)
MAP@5: 0.11576189399080168
Iteration No: 971 ended. Search finished for the next optimal point.
Time taken: 22.8430
Function value obtained: -0.1158
Current minimum: -0.1221
Iteration No: 972 started. Searching for the next optimal point.
[0.018618282767038589, 0.86408366082108901, 0.0019043886938729941, 0.16289254695982283, 0.11120622689403073]
Similarity matrix ready!
Similarity matrix ready!
R_hat done
Shape of final matrix:  (4421, 17296)
MAP@5: 0

Similarity matrix ready!
Similarity matrix ready!
R_hat done
Shape of final matrix:  (4421, 17296)
MAP@5: 0.11319384754580429
Iteration No: 989 ended. Search finished for the next optimal point.
Time taken: 22.5050
Function value obtained: -0.1132
Current minimum: -0.1221
Iteration No: 990 started. Searching for the next optimal point.
[0.012798053076802599, 0.52335282335692124, 0.50864941536982777, 0.0069322717428330644, 0.06578394875999917]
Similarity matrix ready!
Similarity matrix ready!
R_hat done
Shape of final matrix:  (4421, 17296)
MAP@5: 0.11451783156148702
Iteration No: 990 ended. Search finished for the next optimal point.
Time taken: 22.5112
Function value obtained: -0.1145
Current minimum: -0.1221
Iteration No: 991 started. Searching for the next optimal point.
[0.065661163974324657, 0.79310238846222891, 0.46198729018488782, 0.019651902995331819, 0.33273724162296964]
Similarity matrix ready!
Similarity matrix ready!
R_hat done
Shape of final matrix:  (4421, 17296)
MAP@5: 0

## MaxLIPO+TR optimization

In [30]:
def utility_func(icm_w, urm_w, cbf_full_w, slim_w):
    global S_icm, S_urm, S_cbf_full, S_bprslim, urm, tg_playlist, evaluator
    print(icm_w, urm_w, cbf_full_w, slim_w)

    S = S_icm * icm_w + S_urm * urm_w + S_cbf_full * cbf_full_w + S_bprslim * slim_w
    print("Similarity matrix ready!")
    
    S = top_k_filtering(S.transpose(), topK=300).transpose()

    s_norm = S.sum(axis=0)
    s_norm[s_norm==0] = 1
    # Normalize S
    S= S.multiply(csr_matrix(np.reciprocal(s_norm)))
    
    print("Similarity matrix ready!")

    # Keep only the target playlists in the URM
    urm_cleaned = urm[[dataset.get_playlist_index_from_id(x)
                       for x in tg_playlist]]

    # Compute ratings
    R_hat = urm_cleaned.dot(S.tocsc()).tocsr()
    print("R_hat done")

    # Remove the entries in R_hat that are already present in the URM
    urm_cleaned = urm_cleaned[:, [dataset.get_track_index_from_id(x)
                                  for x in tg_tracks]]
    R_hat[urm_cleaned.nonzero()] = 0
    R_hat.eliminate_zeros()

    print("Shape of final matrix: ", R_hat.shape)
    recs = predict(R_hat, list(tg_playlist), list(tg_tracks))
    map_at_five = ev.evaluate_fold(recs)
    return map_at_five

In [31]:
import dlib

# Optimization
# 20190109
# Optimal inputs: [0.19928918511079063, 0.7812370642153202, 0.7506191394524616, 0.3282280765460199]
# Optimal output: 0.11682801779386265
# Optimal inputs: [0.5749155271360308, 0.9274320628644216, 0.9124670208674319, 0.183611133527094]
# Optimal output: 0.12199200784136341
print("Running DLIB optimizer...")
inputs, output = dlib.find_max_global(utility_func,
                                      [0.0, 0.0, 0.0, 0.0],
                                      [1.0, 1.0, 1.0, 1.0],
                                      400)
print("Optimal inputs: {}".format(inputs))
print("Optimal output: {}".format(output))

Running DLIB optimizer...
0.5 0.5 0.5 0.5
Similarity matrix ready!
Similarity matrix ready!
R_hat done
Shape of final matrix:  (4421, 17164)
MAP@5: 0.12108572721103852
0.23327267073693825 0.7626503136151922 0.14639301241805264 0.28180188990451166
Similarity matrix ready!
Similarity matrix ready!
R_hat done
Shape of final matrix:  (4421, 17164)
MAP@5: 0.119961547161276
0.4434393368568707 0.13503727297170132 0.5685008415446849 0.6022759761409715
Similarity matrix ready!
Similarity matrix ready!
R_hat done
Shape of final matrix:  (4421, 17164)
MAP@5: 0.1193025710623544
0.5382985038752768 0.7854426540002342 0.9341968125830351 0.6912492582833162
Similarity matrix ready!
Similarity matrix ready!
R_hat done
Shape of final matrix:  (4421, 17164)
MAP@5: 0.12125235617884375
0.9742896497665218 0.9987349367018152 0.012980778275569137 0.9121755934651533
Similarity matrix ready!
Similarity matrix ready!
R_hat done
Shape of final matrix:  (4421, 17164)
MAP@5: 0.11914423584407789
0.9456690276188229 0.

MAP@5: 0.12190077659654701
0.32434902042069885 0.9873130123061278 0.7887986834701932 0.7631847468228113
Similarity matrix ready!
Similarity matrix ready!
R_hat done
Shape of final matrix:  (4421, 17164)
MAP@5: 0.1213594209454878
0.5684634622615301 0.9278010571019275 0.895861305182007 0.19022084794743419
Similarity matrix ready!
Similarity matrix ready!
R_hat done
Shape of final matrix:  (4421, 17164)
MAP@5: 0.12187966523411015
0.017294244412337238 0.2856447935007376 0.5798141199944188 0.04758646993031961
Similarity matrix ready!
Similarity matrix ready!
R_hat done
Shape of final matrix:  (4421, 17164)
MAP@5: 0.1192852295860669
0.5699750688707869 0.9303851633041764 0.9059408157942853 0.18301945567362587
Similarity matrix ready!
Similarity matrix ready!
R_hat done
Shape of final matrix:  (4421, 17164)
MAP@5: 0.12192339591344364
0.8242143313322102 0.5759925159835985 0.04953616613488182 0.1307786144535652
Similarity matrix ready!
Similarity matrix ready!
R_hat done
Shape of final matrix:  

Similarity matrix ready!
Similarity matrix ready!
R_hat done
Shape of final matrix:  (4421, 17164)
MAP@5: 0.12066425393953142
0.5749156266680706 0.9274320894686592 0.9124669679538574 0.18361121496454216
Similarity matrix ready!
Similarity matrix ready!
R_hat done
Shape of final matrix:  (4421, 17164)
MAP@5: 0.12199200784136341
0.22871198026947046 0.9984662812346909 0.9490095257114031 0.6316418738073558
Similarity matrix ready!
Similarity matrix ready!
R_hat done
Shape of final matrix:  (4421, 17164)
MAP@5: 0.12141446128326963
0.5749154953872517 0.9274320343753398 0.9124669920959393 0.1836110849043868
Similarity matrix ready!
Similarity matrix ready!
R_hat done
Shape of final matrix:  (4421, 17164)
MAP@5: 0.12199200784136341
0.1797600915510819 0.30032347597971765 0.8301455319356662 0.4772302024823512
Similarity matrix ready!
Similarity matrix ready!
R_hat done
Shape of final matrix:  (4421, 17164)
MAP@5: 0.12185855387167355
0.5749155186681002 0.9274320521279475 0.9124669890980074 0.1836

0.5749155271360454 0.9274320628645387 0.9124670208673733 0.18361113352712327
Similarity matrix ready!
Similarity matrix ready!
R_hat done
Shape of final matrix:  (4421, 17164)
MAP@5: 0.12199200784136341
0.3351574302669118 0.7098817841253743 0.6620082529302729 0.7064978235160796
Similarity matrix ready!
Similarity matrix ready!
R_hat done
Shape of final matrix:  (4421, 17164)
MAP@5: 0.1214642237804422
0.5749155271360388 0.9274320628643897 0.9124670208674638 0.1836111335271437
Similarity matrix ready!
Similarity matrix ready!
R_hat done
Shape of final matrix:  (4421, 17164)
MAP@5: 0.12199200784136341
0.3325994293407366 0.9109862189139685 0.8287995558875912 0.9974567110002324
Similarity matrix ready!
Similarity matrix ready!
R_hat done
Shape of final matrix:  (4421, 17164)
MAP@5: 0.12120560959059073
0.5749155271360109 0.9274320628643943 0.9124670208674319 0.183611133527094
Similarity matrix ready!
Similarity matrix ready!
R_hat done
Shape of final matrix:  (4421, 17164)
MAP@5: 0.121992007

R_hat done
Shape of final matrix:  (4421, 17164)
MAP@5: 0.12199200784136341
0.9920968797977492 0.9983358908879543 0.792648295935737 0.09437746890178676
Similarity matrix ready!
Similarity matrix ready!
R_hat done
Shape of final matrix:  (4421, 17164)
MAP@5: 0.12063334087310595
0.5749155271360308 0.9274320628644216 0.9124670208674319 0.183611133527094
Similarity matrix ready!
Similarity matrix ready!
R_hat done
Shape of final matrix:  (4421, 17164)
MAP@5: 0.12199200784136341
0.5592840275477845 0.7210286603968636 0.8276463928318264 0.2941385079181035
Similarity matrix ready!
Similarity matrix ready!
R_hat done
Shape of final matrix:  (4421, 17164)
MAP@5: 0.121236522657016
0.5749155271360308 0.9274320628644216 0.9124670208674319 0.183611133527094
Similarity matrix ready!
Similarity matrix ready!
R_hat done
Shape of final matrix:  (4421, 17164)
MAP@5: 0.12199200784136341
0.43554930794739166 0.9968329743957296 0.9177544999846385 0.8220896938211971
Similarity matrix ready!
Similarity matrix 

Similarity matrix ready!
Similarity matrix ready!
R_hat done
Shape of final matrix:  (4421, 17164)
MAP@5: 0.12184950614491487
0.3615405682804903 0.9737133788582139 0.08936793471309941 0.005557073642069828
Similarity matrix ready!
Similarity matrix ready!
R_hat done
Shape of final matrix:  (4421, 17164)
MAP@5: 0.11788509387016549
0.019687969097976387 0.7055649167660066 0.8445732375289651 0.5345705447806797
Similarity matrix ready!
Similarity matrix ready!
R_hat done
Shape of final matrix:  (4421, 17164)
MAP@5: 0.12103068687325683
0.38662017065315396 0.9972908289057825 0.99570030796443 0.6801025209048377
Similarity matrix ready!
Similarity matrix ready!
R_hat done
Shape of final matrix:  (4421, 17164)
MAP@5: 0.12171982206137419
0.6104301049167999 0.9154488019870323 0.022383504266287064 0.9989726639115017
Similarity matrix ready!
Similarity matrix ready!
R_hat done
Shape of final matrix:  (4421, 17164)
MAP@5: 0.11947598582522845
0.6159221037996068 0.9414189471805652 0.6491022306353876 0.4

MAP@5: 0.12137148458116594
0.02518266454456261 0.9534726618655857 0.6317159417248263 0.6505968474794614
Similarity matrix ready!
Similarity matrix ready!
R_hat done
Shape of final matrix:  (4421, 17164)
MAP@5: 0.1199924602277015
0.6444315837932052 0.9133138668917548 0.9607754988827285 0.8038856249991211
Similarity matrix ready!
Similarity matrix ready!
R_hat done
Shape of final matrix:  (4421, 17164)
MAP@5: 0.12092136017492308
0.10926228271599606 0.31394376243985284 0.8076603199808003 0.360800585413814
Similarity matrix ready!
Similarity matrix ready!
R_hat done
Shape of final matrix:  (4421, 17164)
MAP@5: 0.12181482319234
0.512646809540028 0.8489901589631393 0.5170006929711253 0.4449540892630936
Similarity matrix ready!
Similarity matrix ready!
R_hat done
Shape of final matrix:  (4421, 17164)
MAP@5: 0.12117620447862498
0.0025843313095421736 0.9150498951417768 0.5688727892403788 0.9436519954010438
Similarity matrix ready!
Similarity matrix ready!
R_hat done
Shape of final matrix:  (442

Similarity matrix ready!
Similarity matrix ready!
R_hat done
Shape of final matrix:  (4421, 17164)
MAP@5: 0.12127497549574028
0.47852490382781615 0.6759363040085788 0.8682403290736886 0.1548293326447384
Similarity matrix ready!
Similarity matrix ready!
R_hat done
Shape of final matrix:  (4421, 17164)
MAP@5: 0.12166327376913245
0.4224785281250703 0.5767027339304233 0.9966830762950953 0.5016412435982057
Similarity matrix ready!
Similarity matrix ready!
R_hat done
Shape of final matrix:  (4421, 17164)
MAP@5: 0.12157279650154604
0.5715652101866714 0.8383797019129445 0.4299756410223111 0.8168603027738818
Similarity matrix ready!
Similarity matrix ready!
R_hat done
Shape of final matrix:  (4421, 17164)
MAP@5: 0.12073437382191106
0.5733488199108817 0.9951511561502782 0.21679546903889674 0.41896205052175756
Similarity matrix ready!
Similarity matrix ready!
R_hat done
Shape of final matrix:  (4421, 17164)
MAP@5: 0.12051421247078366
0.48852872743065295 0.9590663444636821 0.8784407725744174 0.318

MAP@5: 0.121673075473121
0.4905482595979381 0.9294841764987534 0.9904138012474808 0.8935392454592085
Similarity matrix ready!
Similarity matrix ready!
R_hat done
Shape of final matrix:  (4421, 17164)
MAP@5: 0.12162406695317844
0.5073938768560015 0.7155446084876355 0.9409024092371889 0.25252025782941784
Similarity matrix ready!
Similarity matrix ready!
R_hat done
Shape of final matrix:  (4421, 17164)
MAP@5: 0.12149212093794796
0.9532371446043012 0.992032790210313 0.9787734853949314 0.11911201452644389
Similarity matrix ready!
Similarity matrix ready!
R_hat done
Shape of final matrix:  (4421, 17164)
MAP@5: 0.12077207268340534
0.61438837091219 0.9750485978539732 0.922405259582511 0.5891282083316673
Similarity matrix ready!
Similarity matrix ready!
R_hat done
Shape of final matrix:  (4421, 17164)
MAP@5: 0.12105858403076254
0.4026466635792736 0.43847292765900436 0.9495205512865144 0.25900404309187497
Similarity matrix ready!
Similarity matrix ready!
R_hat done
Shape of final matrix:  (4421,

Similarity matrix ready!
Similarity matrix ready!
R_hat done
Shape of final matrix:  (4421, 17164)
MAP@5: 0.12082108120334799
0.17651124749547323 0.43746056275523204 0.997141394872809 0.3361334367263755
Similarity matrix ready!
Similarity matrix ready!
R_hat done
Shape of final matrix:  (4421, 17164)
MAP@5: 0.12181331523788022
0.5518446667136861 0.6843548136591396 0.7360623694620851 0.19086355420149623
Similarity matrix ready!
Similarity matrix ready!
R_hat done
Shape of final matrix:  (4421, 17164)
MAP@5: 0.12154640729849994
0.9879263207021827 0.640941892587913 0.6873115217646119 0.9943993675790919
Similarity matrix ready!
Similarity matrix ready!
R_hat done
Shape of final matrix:  (4421, 17164)
MAP@5: 0.11958380456910236
0.4423111377626136 0.5951781896257613 0.7325900838430133 0.652328618721461
Similarity matrix ready!
Similarity matrix ready!
R_hat done
Shape of final matrix:  (4421, 17164)
MAP@5: 0.1214943828696377
0.3133001969255684 0.9087648185734055 0.8859310648342985 0.84708574

In [37]:
# album: 1.6 artist: 1.5 urm: 1.0
# Evaluating S_ICM only...
# MAP@5: 0.10570609967578998
# Evaluating S_URM only...
# MAP@5: 0.08959134434140079
# Evaluating S_CBF_FULL only...
# MAP@5: 0.12028274146120818
# Evaluating SLIM_BPR only...
# MAP@5: 0.11036869486541527
# Single model evaluation
print("Evaluating S_ICM only...")
utility_func(1, 0, 0, 0)

print("Evaluating S_URM only...")
utility_func(0, 1, 0, 0)

print("Evaluating S_CBF_FULL only...")
utility_func(0, 0, 1, 0)

print("Evaluating SLIM_BPR only...")
utility_func(0, 0, 0, 1)

Evaluating S_ICM only...
1 0 0 0
Similarity matrix ready!
Similarity matrix ready!
R_hat done
Shape of final matrix:  (4421, 17164)
MAP@5: 0.10535474628666233
Evaluating S_URM only...
0 1 0 0
Similarity matrix ready!
Similarity matrix ready!
R_hat done
Shape of final matrix:  (4421, 17164)
MAP@5: 0.08959134434140079
Evaluating S_CBF_FULL only...
0 0 1 0
Similarity matrix ready!
Similarity matrix ready!
R_hat done
Shape of final matrix:  (4421, 17164)
MAP@5: 0.11978737842117212
Evaluating SLIM_BPR only...
0 0 0 1
Similarity matrix ready!
Similarity matrix ready!
R_hat done


/media/leonardoarcari/Data/Home/Development/python_projects/recsys_challenge/lib/python3.6/site-packages/scipy/sparse/compressed.py:742: SparseEfficiencyWarning: Changing the sparsity structure of a csr_matrix is expensive. lil_matrix is more efficient.
  SparseEfficiencyWarning)


Shape of final matrix:  (4421, 17164)
MAP@5: 0.11036869486541527


0.11036869486541527

# Final submission

In [38]:
%autoreload 2

from src.utils.loader import *
print("started")
print("hey")
dataset = Dataset(load_tags=True,
                  filter_tag=False,
                  weight_tag=False)
dataset.set_track_attr_weights_2(1.5, 1.6, 0.0, 0.0, 0.0,
                                 1.0, 0.0, 0.0, 0.0)
urm = dataset.build_train_matrix()
tg_playlist = list(dataset.target_playlists.keys())
tg_tracks = list(dataset.target_tracks.keys())

started
hey
File found, retrieving urm from it.
Load from file takes 1.65 seconds


In [39]:
from src.ML.BPRSLIM import BPRSLIM
from src.utils.matrix_utils import applyTfIdf, top_k_filtering

import os
dataset.set_track_attr_weights_2(1, 1, 0, 0, 0, 0, 0, 0, 0)

print('Building the ICM for SLIMBPR...')
icm = dataset.build_icm()

icm_tag = dataset.build_tags_matrix()

tags = applyTfIdf(icm_tag)

# Before stacking tags with the rest of the ICM, we keep only
# the top K tags for each item. This way we try to reduce the
# natural noise added by such sparse features.
tags = top_k_filtering(tags.transpose(), topK=55).transpose()
tags.data = np.ones_like(tags.data)

# stack all
icm = vstack([icm, tags], format='csr')

Building the ICM for SLIMBPR...


In [40]:
print("Training SLIM BPR...")
slim = BPRSLIM(epochs=30,
               epochMultiplier=1.0,
               sgd_mode='rmsprop',
               learning_rate=5e-05,
               topK=300,
               urmSamplingChances=1 / 5,
               icmSamplingChances=4 / 5)

# If present, load SLIM BPR parameters from disk
w_path = './data/slim_bpr_full_parameters.npz'
if os.path.isfile(w_path):
    W = load_sparse_matrix(w_path)
    slim.W = W.tocsr()
    S_bprslim = W.tocsr()
    print("BPR SLIM parameters loaded...")
else:
    slim.fit(urm.tocsr(),
             icm.tocsr(),
             tg_playlist,
             tg_tracks,
             dataset)
    S_bprslim = slim.getParameters()

print("Cleaning S_bprslim")
S_bprslim = S_bprslim[:,[dataset.get_track_index_from_id(x) for x in tg_tracks]]

print("Filtering...")
# keep only 100 elements
S_bprslim = top_k_filtering(S_bprslim.transpose(), 100).transpose()

print("Normalizing...")
# normalize
s_norm = S_bprslim.sum(axis=0)
s_norm[s_norm==0] = 1
print("S done")

S_bprslim = S_bprslim.multiply(csr_matrix(np.reciprocal(s_norm)))

Training SLIM BPR...
BPR SLIM parameters loaded...
Cleaning S_bprslim
Filtering...
Normalizing...
S done


In [22]:
# If not present, save SLIM BPR parameters to disk
w_path = './data/slim_bpr_full_parameters.npz'
if not os.path.isfile(w_path):
    save_sparse_matrix(w_path, slim.getParameters())

In [41]:
print("ok")
# Initiliaze Models
dataset.set_track_attr_weights_2(1.8272547701242121, 1.9097988118972682, 0.0, 0.0, 0.0,
                                 1.0, 0.0, 0.0, 0.0)
cbf = ContentBasedFiltering()
cbf.fit(urm * 0.7840544888432146,
        list(tg_playlist),
        list(tg_tracks),
        dataset, 
        topK_tag=55)

S_cbf_full = cbf.S

ok
CBF started
Build tags matrix and apply TFIDF...
Running 4 workers...
[ 7231 ] Building cosine similarity matrix for [0, 1000)...
[ 7232 ] Building cosine similarity matrix for [8048, 9048)...
[ 7233 ] Building cosine similarity matrix for [16096, 17096)...
[ 7234 ] Building cosine similarity matrix for [24144, 25144)...
[ 7232 ] Building cosine similarity matrix for [9048, 10048)...
[ 7231 ] Building cosine similarity matrix for [1000, 2000)...
[ 7233 ] Building cosine similarity matrix for [17096, 18096)...
[ 7234 ] Building cosine similarity matrix for [25144, 26144)...
[ 7231 ] Building cosine similarity matrix for [2000, 3000)...
[ 7232 ] Building cosine similarity matrix for [10048, 11048)...
[ 7233 ] Building cosine similarity matrix for [18096, 19096)...
[ 7234 ] Building cosine similarity matrix for [26144, 27144)...
[ 7232 ] Building cosine similarity matrix for [11048, 12048)...
[ 7233 ] Building cosine similarity matrix for [19096, 20096)...
[ 7231 ] Building cosine simi

In [44]:
S_urm = ibs(urm, dataset)

Running 4 workers...
[ 7505 ] Building cosine similarity matrix for [0, 1000)...
[ 7506 ] Building cosine similarity matrix for [8048, 9048)...
[ 7507 ] Building cosine similarity matrix for [16096, 17096)...
[ 7508 ] Building cosine similarity matrix for [24144, 25144)...
[ 7506 ] Building cosine similarity matrix for [9048, 10048)...
[ 7505 ] Building cosine similarity matrix for [1000, 2000)...
[ 7507 ] Building cosine similarity matrix for [17096, 18096)...
[ 7508 ] Building cosine similarity matrix for [25144, 26144)...
[ 7506 ] Building cosine similarity matrix for [10048, 11048)...
[ 7505 ] Building cosine similarity matrix for [2000, 3000)...
[ 7507 ] Building cosine similarity matrix for [18096, 19096)...
[ 7508 ] Building cosine similarity matrix for [26144, 27144)...
[ 7506 ] Building cosine similarity matrix for [11048, 12048)...
[ 7505 ] Building cosine similarity matrix for [3000, 4000)...
[ 7506 ] Building cosine similarity matrix for [12048, 13048)...
[ 7505 ] Building 

In [42]:
dataset.set_track_attr_weights_2(1.8272547701242121, 1.9097988118972682, 0.0, 0.0, 0.0,
                                 1.0, 0.0, 0.0, 0.0)
S_icm = icm_sim(dataset.build_icm(), dataset)

Running 4 workers...
[ 7395 ] Building cosine similarity matrix for [8048, 9048)...
[ 7394 ] Building cosine similarity matrix for [0, 1000)...
[ 7396 ] Building cosine similarity matrix for [16096, 17096)...
[ 7397 ] Building cosine similarity matrix for [24144, 25144)...
[ 7394 ] Building cosine similarity matrix for [1000, 2000)...
[ 7395 ] Building cosine similarity matrix for [9048, 10048)...
[ 7396 ] Building cosine similarity matrix for [17096, 18096)...
[ 7397 ] Building cosine similarity matrix for [25144, 26144)...
[ 7394 ] Building cosine similarity matrix for [2000, 3000)...
[ 7396 ] Building cosine similarity matrix for [18096, 19096)...
[ 7395 ] Building cosine similarity matrix for [10048, 11048)...
[ 7397 ] Building cosine similarity matrix for [26144, 27144)...
[ 7396 ] Building cosine similarity matrix for [19096, 20096)...
[ 7395 ] Building cosine similarity matrix for [11048, 12048)...
[ 7394 ] Building cosine similarity matrix for [3000, 4000)...
[ 7397 ] Building 

In [12]:
dataset.set_playlist_attr_weights(1, 1, 1, 0, 0)
ucm = dataset.build_ucm()
S_user = ubs(urm, ucm, dataset)

Running 4 workers...
[ 9021 ] Building cosine similarity matrix for [0, 250)...
[ 9022 ] Building cosine similarity matrix for [8048, 8298)...
[ 9023 ] Building cosine similarity matrix for [16096, 16346)...
[ 9024 ] Building cosine similarity matrix for [24144, 24394)...
[ 9023 ] Building cosine similarity matrix for [16346, 16596)...
[ 9022 ] Building cosine similarity matrix for [8298, 8548)...
[ 9021 ] Building cosine similarity matrix for [250, 500)...
[ 9024 ] Building cosine similarity matrix for [24394, 24644)...
[ 9023 ] Building cosine similarity matrix for [16596, 16846)...
[ 9021 ] Building cosine similarity matrix for [500, 750)...
[ 9022 ] Building cosine similarity matrix for [8548, 8798)...
[ 9024 ] Building cosine similarity matrix for [24644, 24894)...
[ 9023 ] Building cosine similarity matrix for [16846, 17096)...
[ 9022 ] Building cosine similarity matrix for [8798, 9048)...
[ 9024 ] Building cosine similarity matrix for [24894, 25144)...
[ 9021 ] Building cosine s

[ 9023 ] Building cosine similarity matrix for [24096, 24144)...
[ 9023 ] Building cosine similarity matrix for [32192, 32195)...
[ 9024 ] Building cosine similarity matrix for [32144, 32192)...
[ 9022 ] Building cosine similarity matrix for [16048, 16096)...
[ 9021 ] Building cosine similarity matrix for [8000, 8048)...
S done
Similarity matrix ready!


### Matrix Factorization

In [26]:
from src.CBF.CBF_MF import ContentBasedFiltering as CBF
from src.MF.MF_BPR.MF_BPR import MF_BPR

# Init ICM params
dataset.set_track_attr_weights_2(1.5, 1.6, 0, 0, 0.0,
                                 1.0, 0, 0.0, 0.0)
cbf = CBF()
    
cbf.fit(urm, tg_playlist,
        tg_tracks,
        dataset)

# get R_hat
R_hat_aug = cbf.getR_hat()
print(R_hat_aug.nnz)

mf = MF_BPR()

CBF started
Build tags matrix and apply TFIDF...
Running 4 workers...
[ 13078 ] Building cosine similarity matrix for [0, 1000)...
[ 13079 ] Building cosine similarity matrix for [25000, 26000)...
[ 13080 ] Building cosine similarity matrix for [50000, 51000)...
[ 13081 ] Building cosine similarity matrix for [75000, 76000)...
[ 13078 ] Building cosine similarity matrix for [1000, 2000)...
[ 13079 ] Building cosine similarity matrix for [26000, 27000)...
[ 13080 ] Building cosine similarity matrix for [51000, 52000)...
[ 13081 ] Building cosine similarity matrix for [76000, 77000)...
[ 13079 ] Building cosine similarity matrix for [27000, 28000)...
[ 13078 ] Building cosine similarity matrix for [2000, 3000)...
[ 13080 ] Building cosine similarity matrix for [52000, 53000)...
[ 13081 ] Building cosine similarity matrix for [77000, 78000)...
[ 13079 ] Building cosine similarity matrix for [28000, 29000)...
[ 13078 ] Building cosine similarity matrix for [3000, 4000)...
[ 13080 ] Buildin

/media/leonardoarcari/Data/Home/Development/python_projects/recsys_challenge/lib/python3.6/site-packages/scipy/sparse/compressed.py:742: SparseEfficiencyWarning: Changing the sparsity structure of a csr_matrix is expensive. lil_matrix is more efficient.
  SparseEfficiencyWarning)


Shape of final matrix:  (57561, 100000)
1268715


In [27]:
w_path = './data/mf_bpr_full_W.npz'
h_path = './data/mf_bpr_full_H.npz'
if os.path.isfile(w_path) and os.path.isfile(h_path):
    W = load_sparse_matrix(w_path)
    H = load_sparse_matrix(h_path)
    mf.W = W.tocsr()
    mf.H = H.tocsr()
    print("BPR MF parameters loaded...")
else:
    mf.fit(R_hat_aug, dataset, list(tg_playlist), list(tg_tracks), n_epochs=1, no_components=500, epoch_multiplier=6, l_rate=1e-2, use_icm=False)

Processed 50000 ( 0.66% ) in 9.09 seconds. Sample per second: 5501
Processed 100000 ( 1.31% ) in 13.95 seconds. Sample per second: 4356
Processed 150000 ( 1.97% ) in 19.08 seconds. Sample per second: 3652
Processed 200000 ( 2.63% ) in 20.35 seconds. Sample per second: 3260
Processed 250000 ( 3.28% ) in 21.72 seconds. Sample per second: 3022
Processed 300000 ( 3.94% ) in 22.71 seconds. Sample per second: 2865
Processed 350000 ( 4.60% ) in 23.00 seconds. Sample per second: 2756
Processed 400000 ( 5.25% ) in 22.51 seconds. Sample per second: 2675
Processed 450000 ( 5.91% ) in 23.10 seconds. Sample per second: 2615
Processed 500000 ( 6.57% ) in 22.69 seconds. Sample per second: 2568
Processed 550000 ( 7.23% ) in 23.64 seconds. Sample per second: 2527
Processed 600000 ( 7.88% ) in 23.17 seconds. Sample per second: 2498
Processed 650000 ( 8.54% ) in 22.83 seconds. Sample per second: 2473
Processed 700000 ( 9.20% ) in 23.35 seconds. Sample per second: 2453
Processed 750000 ( 9.85% ) in 22.87 

Processed 5850000 ( 76.85% ) in 22.73 seconds. Sample per second: 2236
Processed 5900000 ( 77.51% ) in 23.35 seconds. Sample per second: 2235
Processed 5950000 ( 78.16% ) in 23.04 seconds. Sample per second: 2235
Processed 6000000 ( 78.82% ) in 22.61 seconds. Sample per second: 2235
Processed 6050000 ( 79.48% ) in 23.22 seconds. Sample per second: 2235
Processed 6100000 ( 80.13% ) in 22.89 seconds. Sample per second: 2235
Processed 6150000 ( 80.79% ) in 23.56 seconds. Sample per second: 2234
Processed 6200000 ( 81.45% ) in 23.32 seconds. Sample per second: 2234
Processed 6250000 ( 82.10% ) in 22.94 seconds. Sample per second: 2234
Processed 6300000 ( 82.76% ) in 23.60 seconds. Sample per second: 2234
Processed 6350000 ( 83.42% ) in 23.24 seconds. Sample per second: 2233
Processed 6400000 ( 84.07% ) in 22.94 seconds. Sample per second: 2233
Processed 6450000 ( 84.73% ) in 23.52 seconds. Sample per second: 2233
Processed 6500000 ( 85.39% ) in 23.24 seconds. Sample per second: 2233
Proces

NameError: name '_build_similarity' is not defined

In [30]:
S = csr_matrix(mf.m_dot_chunked(
            mf.H[[mf.dataset.get_track_index_from_id(x)
                    for x in mf.tr_id_list]],
            mf.H.transpose(),
            topK=100))
print("S done")

# Normalize S
s_norm = S.sum(axis=1)
S = S.multiply(csr_matrix(np.reciprocal(s_norm)))
S_mf = S


Computing dot product for chunk [0, 1000)...
Computing dot product for chunk [1000, 2000)...
Computing dot product for chunk [2000, 3000)...
Computing dot product for chunk [3000, 4000)...
Computing dot product for chunk [4000, 5000)...
Computing dot product for chunk [5000, 6000)...
Computing dot product for chunk [6000, 7000)...
Computing dot product for chunk [7000, 8000)...
Computing dot product for chunk [8000, 9000)...
Computing dot product for chunk [9000, 10000)...
Computing dot product for chunk [10000, 11000)...
Computing dot product for chunk [11000, 12000)...
Computing dot product for chunk [12000, 13000)...
Computing dot product for chunk [13000, 14000)...
Computing dot product for chunk [14000, 15000)...
Computing dot product for chunk [15000, 16000)...
Computing dot product for chunk [16000, 17000)...
Computing dot product for chunk [17000, 18000)...
Computing dot product for chunk [18000, 19000)...
Computing dot product for chunk [19000, 20000)...
Computing dot product 

In [32]:
# If not present, save MF BPR parameters to disk
w_path = './data/mf_bpr_full_W.npz'
h_path = './data/mf_bpr_full_H.npz'
if not os.path.isfile(w_path):
    save_sparse_matrix(w_path, csr_matrix(mf.W))
if not os.path.isfile(h_path):
    save_sparse_matrix(h_path, csr_matrix(mf.H))

In [45]:
# Best params 0.53119031862654198, 0.8622528471283929, 0.449568357270693, 0.84725181341512967]
# [0.24995989924853496, 0.35681721890423695, 0.12389358733625891, 0.78110404084094665]
# weights = [0.53, 0.86, 0.45, 0.85]
# S = S_icm * 0.25550064193970895 + S_urm * 0.09295686755690992 + S_cbf_full * 0.73164854390113454 + S_bprslim * 0.2395213452495030 + S_user * 0.99898125828950768
S = S_icm * 0.010884732743304261 + S_urm * 0.02719900735954462 + S_cbf_full * 0.048684679013303325 + S_bprslim * 0.022415631857985566
# S = S_icm * 0.19928918511079063 + S_urm * 0.7812370642153202 + S_cbf_full * 0.7506191394524616 + S_bprslim * 0.3282280765460199
# S = S_icm * 0.5677248728726197 + S_urm * 0.5518258064369265 + S_cbf_full * 0.9622603220708489 + S_bprslim * 0.11921505196603724 + S_mf.transpose() * 0.12842883273924657
print("Similarity matrix ready!")

S = top_k_filtering(S.transpose(), topK=300).transpose()

s_norm = S.sum(axis=0)
s_norm[s_norm==0] = 1
# Normalize S
S = S.multiply(csr_matrix(np.reciprocal(s_norm)))

# Keep only the target playlists in the URM
urm_cleaned = urm[[dataset.get_playlist_index_from_id(x)
                   for x in tg_playlist]]

# Compute ratings
R_hat = urm_cleaned.dot(S.tocsc()).tocsr()
print("R_hat done")

# Remove the entries in R_hat that are already present in the URM
urm_cleaned = urm_cleaned[:, [dataset.get_track_index_from_id(x)
                              for x in tg_tracks]]
R_hat[urm_cleaned.nonzero()] = 0
R_hat.eliminate_zeros()

print("Shape of final matrix: ", R_hat.shape)
recs = predict(R_hat, list(tg_playlist), list(tg_tracks))

Similarity matrix ready!
R_hat done
Shape of final matrix:  (10000, 32195)


In [46]:
with open('submission_ensemble.csv', mode='w', newline='') as out:
    fieldnames = ['playlist_id', 'track_ids']
    writer = csv.DictWriter(out, fieldnames=fieldnames, delimiter=',')
    writer.writeheader()
    for k in tg_playlist:
        track_ids = ''
        for r in recs[k]:
            track_ids = track_ids + r + ' '
        writer.writerow({'playlist_id': k,
                         'track_ids': track_ids[:-1]})